In [1]:
# import the neccessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Assignment - Load Forecasting

predicting future electricity load consumption, from historical data.
Various factors can also be taken into consideration while forecasting, for instance, weather conditions can be sent as an input in the mathematical foreteller function to improve accuracy parameters in certain scenarios.

In [2]:
# load the dataset

train_ds = pd.read_csv('assignment-data.csv',
                      index_col=['datetime'], parse_dates=['datetime'])

FileNotFoundError: ignored

In [ ]:
# lets explore our dataset

train_ds.head()

# Data Cleaning in Time Series

We need to remove the seasonality, trend (upwards/downwards), and make it stationary 
However, the data was already stationary

In [ ]:
# checking the number of observations in our dataset

def explore_ds(df):
    
    print(f'Total No. of rows: {df.shape[0]}')
    print('-------------------------------')
    print(df.describe())
    print('-------------------------------')
    print(df.info())


In [ ]:
explore_ds(train_ds)

In [ ]:
# seeing above dataset, we can drop Unnamed:0 as its indexing and not useful feature for us
# we can also drop date as its derived from datetime column

train_ds.drop(['Unnamed: 0', 'date'], axis=1, inplace=True)

In [ ]:
# To understand and visualize our dataset better, lets try to resample the data hourly

train_ds_hourly = train_ds.resample('1H').mean() # taking average of 15 min every hour 
train_ds_hourly.head(3)

In [ ]:
# datetime columns has many useful features such as day, month, week etc which can be used for EDA purpose

# train_ds['Year'] = train_ds['datetime'].apply(lambda x: x.year)
# train_ds['Month'] = train_ds['datetime'].apply(lambda x: x.month)
# train_ds['Day'] = train_ds['datetime'].apply(lambda x: x.day)
# train_ds['Weekday'] = train_ds['datetime'].apply(lambda x: x.day_name())
# train_ds['Weeknumber'] = train_ds['datetime'].apply(lambda x: x.weekofyear)
# train_ds['Hour'] = train_ds['datetime'].apply(lambda x: x.hour)
# train_ds['Minute'] = train_ds['datetime'].apply(lambda x: x.minute)


# Tasks:

* Task 1 - Do a short term forecast for the day 14 December 2020 in the frequency of 15 minutes.
* Task 2 - Do forecast for the rows where load values are NaN.

In [ ]:
# Checking for any missing values

train_ds.isna().sum()

* As we see above we have 1440 nan values in load, which apparently becomes our task 2 where we need to predict these values, we will get rid of NaN values

In [ ]:
# including all the load values without NaN

train_ds = train_ds[~train_ds['load'].isna()]

# Lets Visualize our dataset

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, dpi=120, figsize=(10,6))
def visualize_df(df):
    for i, ax in enumerate(axes.flatten()):
        data = df[df.columns[i]]
        ax.plot(data, color='red', linewidth=1)
        # Decorations
        ax.set_title(df.columns[i])
        ax.xaxis.set_ticks_position('none')
        ax.yaxis.set_ticks_position('none')
        ax.spines["top"].set_alpha(0)
        ax.tick_params(labelsize=6)

    plt.tight_layout();
    
visualize_df(train_ds)

We can clearly observe the seasonal changes yearwise in the data pattern

In addition to temperature information, I added season information, which is a time-series factor that affects temperature (especially outside).



# Making our time series more stationary,
In the below code, i have directly done dicky fuller test to check if the time series if stationary or not.
However for better model, we can further make our time series more stationary as we can see the seasonal pattern clearly
in most of the features.

In [ ]:
# checking if our time series is stationary, incase not we need to make it stationary before moving forward

for i in range(len(train_ds.columns)):
  result = adfuller(train_ds[train_ds.columns[i]])

  if result[1] > 0.05 :
    print('{} - Series is not Stationary'.format(train_ds.columns[i]))
  else:
    print('{} - Series is Stationary'.format(train_ds.columns[i]))

In [ ]:
# granger casuality (granger causs) to check if my one value affects another using p value
# simply saying if one time series cause another time series
# we calculate t test, F test

max_lags=8 # defining lag value
y='load' # comparing how load timeseries can cause another time series

In [ ]:
df = train_ds

In [ ]:
for i in range(len(df.columns)-1):
  results=grangercausalitytests(df[[y,df.columns[i+1]]], max_lags, verbose=False)
  p_values=[round(results[i+1][0]['ssr_ftest'][1],4) for i in range(max_lags)]
  print('Column - {} : P_Values - {}'.format(df.columns[i+1],p_values))

If we see above, the null hypothesis is true for apparent_tempearture as its greater than 0.05 (5% significant).
we will use remaining factors as they seem to be more relevant to the load

we will build linear model with load lag for all the factors

In [ ]:
df=train_ds

In [ ]:
# I am considering the below factors as per clue Rainfall effects can have direct load consumption

df_input=df[['load','temperature', 'wind_speed','cloud_cover']]
# df_input=df

In [ ]:
# its time to split our dataset, as its time series data we can't do train test split
# so we will take out 90% data for training and remaining 10% for test

df_train = df_input[:int(0.9*(len(df_input)))] # 90% for training
df_test = df_input[int(0.9*(len(df_input))):] # 10% for test from the last

In [ ]:
print('training dataset', df_train.shape)
print('test dataset', df_test.shape)

In [ ]:
# we will build VAR model, with AIC and BIC for model selection

# here we will see which lags to consider which give us the lowest AIC and BIC value, 
# then with minimum lag we can fit the model

model = VAR(df_train)
for i in range(10):
    results = model.fit(i+1)
    print('Order = ', i+1)
    print('AIC: ', results.aic)
    print('BIC: ', results.bic)

In [ ]:
# to pick the lowest once
model.select_order(10).summary()

# Found this appraoch not much suitable as its required lots of data cleaning & Time Taking

# Evaluation Metrics

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Using Facebook Prophet Library

In [ ]:
# Using for Prophet
train_ds_org = pd.read_csv('assignment-data.csv',
                       parse_dates=['datetime'])
train_ds_org.drop(['Unnamed: 0', 'date'], axis=1, inplace=True)

In [ ]:
from fbprophet import Prophet

In [ ]:
model = Prophet(interval_width = 0.95)
# model.add_regressor('temperature', mode='multiplicative')
# model.add_regressor('wind_speed', mode='multiplicative')
# model.add_regressor('cloud_cover', mode='multiplicative')

In [ ]:
model = Prophet(
    growth='linear',
    n_changepoints=10,
    changepoint_range=0.8,
    yearly_seasonality='auto',
    weekly_seasonality='auto',
    daily_seasonality='auto',
    seasonality_mode='additive',
    seasonality_prior_scale=10.0,
    changepoint_prior_scale=0.05,
    interval_width=0.8)

In [ ]:
train_ds_org.drop(['apparent_temperature','temperature', 'humidity',
       'dew_point', 'wind_speed', 'cloud_cover'], axis=1, inplace=True)

In [ ]:
# removing nan data, can also do using dropna()
# train_ds_org = train_ds_org[~train_ds_org['load'].isna()]

In [ ]:
train_ds_org.info()

In [ ]:
train_ds_new = train_ds_org[['datetime', 'load', 'temperature','wind_speed', 'cloud_cover']].rename({
    'datetime':'ds',
    'load':'y'
}, axis='columns')

In [ ]:
train_ds_new

In [ ]:
train = train_ds_new[:(int(0.9*len(train_ds_new)))]

In [ ]:
test = train_ds_new[(int(0.9*len(train_ds_new))):]

In [ ]:
m = model.fit(train)

In [ ]:
# future = model.make_future_dataframe(periods=1000, freq='15min')
# forecast = model.predict(future)
forecast = model.predict(df=test)
forecast.head()

In [ ]:
model.plot(forecast)

In [ ]:
model.plot_components(forecast)

# Prediction Test

In [ ]:
# Testing on Unseen data
mean_absolute_percentage_error(y_true=test['y'],
                   y_pred=forecast['yhat'])

# Task 1 & 2: Predicting output for next 2 days & Missing NaN values

**I have drop columns again as running the model again for the output**

In [ ]:
train_ds_org

In [ ]:
train_ds_2 = train_ds_org.rename({'datetime':'ds',
                                            'load':'y'
                                           },axis=1)

In [ ]:
model.fit(train_ds_2)

In [ ]:
future_fcst = model.make_future_dataframe(periods=200, freq='15min')
forecast2 = model.predict(future_fcst)

In [ ]:
forecast2

In [ ]:
train_ds_org.shape

In [ ]:
train_ds_org['load'][:103392]

In [ ]:
# getting next 2 days forecast
forecast2[['ds','yhat']][-200:].to_csv('output_task1.csv', header=True, index=False)

In [ ]:
task2 = train_ds_org[train_ds_org['load'].isna()].reset_index(drop=True)

In [ ]:
task2 = task2.rename({'datetime':'ds','load':'y'
                                           },axis=1)

In [ ]:
task2

In [ ]:
# performing join on forecast2
final2 = task2.merge(forecast2, on='ds', how='inner')
final2 = final2[['ds','yhat']]
final2 = final2.rename({'ds':'datetime','yhat':'load'},axis=1)

In [ ]:
final2.to_csv('output_task2.csv', header=True, index=False)